<a href="https://colab.research.google.com/github/OsvaldoMS1982/TFM-NLP2SQL/blob/Fase-1/Fase1_EvaluacionDeModelosPArte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1. Instalar Dependencias

In [ ]:
!pip install sacrebleu datasets torch tabulate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Montar Drive

In [ ]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta del dataset Spider en Google Drive
SPIDER_PATH = "/content/drive/My Drive/spider"

# Verificar que los archivos están en la ubicación correcta
print("Archivos en Spider:", os.listdir(SPIDER_PATH))


Mounted at /content/drive
Archivos en Spider: ['train_gold.sql', 'dev_gold.sql', 'dev.json', 'train_others.json', 'train_spider.json', 'tables.json', 'README.txt', 'test_tables.json', 'test.json', 'test_gold.sql', '.DS_Store', 'test_database', 'database', 'train_spider_fixed.json']


Cargar Modelos y Tokenizers

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# 📌 Lista de modelos a evaluar
models_to_test = {
    "T5-LM-Large-text2sql-spider": "/content/drive/My Drive/spider_models/T5-LM-Large-text2sql-spider",
    "T5-Small-NL2SQL": "/content/drive/My Drive/spider_models/T5-Small-NL2SQL",
    "BART-LARGE-NL2SQL": "/content/drive/My Drive/spider_models/BART-LARGE-NL2SQL"
}

# 📌 Diccionario para almacenar modelos y tokenizers
models_dict = {}

for model_name, model_path in models_to_test.items():
    try:
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        models_dict[model_name] = {"model": model, "tokenizer": tokenizer}
        print(f"✅ {model_name} cargado correctamente.")
    except Exception as e:
        print(f"❌ Error al cargar {model_name}: {e}")


✅ T5-LM-Large-text2sql-spider cargado correctamente.
✅ T5-Small-NL2SQL cargado correctamente.
✅ BART-LARGE-NL2SQL cargado correctamente.


Generar Predicciones SQL para Validación

In [ ]:
import json

SPIDER_PATH = "/content/drive/My Drive/spider"

# 📌 Cargar datos de validación de Spider
with open(f"{SPIDER_PATH}/dev.json", "r") as f:
    val_data = json.load(f)

# 📌 Extraer preguntas y sus SQL correctos
questions = [item["question"] for item in val_data]
true_sql = [item["query"] for item in val_data]

# 📌 Función para generar SQL con cada modelo
def generate_sql(model, tokenizer, question):
    input_text = f"Translate to SQL: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")

    output = model.generate(
        **inputs, max_length=128, temperature=0.7, top_p=0.9, num_return_sequences=1, repetition_penalty=1.2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 📌 Generar consultas SQL con cada modelo
predictions = {}
for model_name, data in models_dict.items():
    print(f"\n🚀 Generando SQL con {model_name}...\n")
    model = data["model"]
    tokenizer = data["tokenizer"]
    predictions[model_name] = [generate_sql(model, tokenizer, q) for q in questions]



🚀 Generando SQL con T5-LM-Large-text2sql-spider...



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



🚀 Generando SQL con T5-Small-NL2SQL...


🚀 Generando SQL con BART-LARGE-NL2SQL...



/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1528: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Both `max_new_tokens` (=128) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=128) seem to have been s

Calcular BLEU Score y Exact Match (EM)

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00


In [ ]:
#from datasets import load_metric
import evaluate
from tabulate import tabulate

# 📌 Cargar métricas BLEU y Exact Match (EM)
bleu_metric = evaluate.load("sacrebleu")

def exact_match(pred, true):
    return int(pred.strip().lower() == true.strip().lower())

# 📌 Evaluar cada modelo
results = []

for model_name in models_dict.keys():
    pred_sql = predictions[model_name]

    # 📌 Calcular BLEU
    bleu_score = bleu_metric.compute(predictions=pred_sql, references=[[x] for x in true_sql])["score"]

    # 📌 Calcular Exact Match (EM)
    em_score = sum(exact_match(pred_sql[i], true_sql[i]) for i in range(len(true_sql))) / len(true_sql)

    results.append([model_name, round(bleu_score, 2), round(em_score * 100, 2)])

# 📌 Mostrar Resultados en Tabla
print("\n📊 **Comparación de Métricas**\n")
print(tabulate(results, headers=["Modelo", "BLEU Score", "Exact Match (%)"], tablefmt="grid"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



📊 **Comparación de Métricas**

+-----------------------------+--------------+-------------------+
| Modelo                      |   BLEU Score |   Exact Match (%) |
+=============================+==============+===================+
| T5-LM-Large-text2sql-spider |        38.05 |              3.48 |
+-----------------------------+--------------+-------------------+
| T5-Small-NL2SQL             |        21.35 |              1.64 |
+-----------------------------+--------------+-------------------+
| BART-LARGE-NL2SQL           |        32.86 |              2.51 |
+-----------------------------+--------------+-------------------+
